In [ ]:
# Instalar os pacotes necessários
try:
    import pymongo
except ImportError:
    %pip install pymongo
    import pymongo

try:
    import pandas as pd
except ImportError:
    %pip install pandas
    import pandas as pd

In [ ]:
client = pymongo.MongoClient('mongodb://localhost:27017/')
db = client['dadosVivamente']
collection = db['dadosSemFiltros']

In [ ]:
# Verificando se a coleção existe
if 'dadosComFiltrosIniciais' in db.list_collection_names():
    print('A coleção já existe')
    collection = db['dadosComFiltrosIniciais']
else:
    pipeline1 = [
        {
            '$match': {
                'posts': {
                    '$exists': True,
                    '$ne': []
                }
            }
        },
        {
            '$match': {
                'autoriza': 'S'
            }
        },
        {
            '$match': {
                'public_profile.locale': 'pt_BR'
            }
        },
        {
            '$out': 'dadosComFiltrosIniciais'
        }
    ]
    collection.aggregate(pipeline1)
    collection = db['dadosComFiltrosIniciais']

In [ ]:
# Pegar apenas os documentos que possuem a data de criação do primeiro post com pelo 7 anos de diferença da data de criação do último post
if 'posts7anos2anos' in db.list_collection_names():
    print('A coleção já existe')
    collection = db['posts7anos2anos']
else:
    # 7 anos em milissegundos
    sete_anos_millis = 7 * 365.25 * 24 * 60 * 60 * 1000

    # 2 anos em milissegundos
    dois_anos_millis = 2 * 365.25 * 24 * 60 * 60 * 1000

    pipeline2 = [
        {
            '$addFields': {
                'firstPostDate': {
                    '$toLong': {
                        '$arrayElemAt': [
                            '$posts.created_time', -1
                        ]
                    }
                },
                'lastPostDate': {
                    '$toLong': {
                        '$arrayElemAt': [
                            '$posts.created_time', 0
                        ]
                    }
                }
            }
        },
        {
            '$addFields': {
                'diff': {
                    '$subtract': [
                        '$lastPostDate', '$firstPostDate'
                    ]
                }
            }
        },
        {
            '$match': {
                '$and': [
                    {'diff': {'$gte': dois_anos_millis}},
                    {'diff': {'$lte': sete_anos_millis}}
                ]
            }
        },
        {
            '$out': 'posts7anos2anos'
        }
    ]
    collection.aggregate(pipeline2)
    collection = db['posts7anos2anos']

In [ ]:
if 'postsComBDIAndInfos' in db.list_collection_names():
    print('A coleção já existe')
    collection = db['postsComBDIAndInfos']
else:
    pipeline3 = [
        {
            '$unwind': '$posts'
        }, {
            '$project': {
                '_id': '$posts._id',
                'idade': 1,
                'sexo': 1,
                'id_usuario': 1,
                'nivel': {
                    '$let': {
                        'vars': {
                            'dividedValue': {
                                '$divide': [
                                    {
                                        '$arrayElemAt': [
                                            '$respostas.nivel', 0
                                        ]
                                    }, 21
                                ]
                            }
                        },
                        'in': {
                            '$cond': {
                                'if': {
                                    '$gte': [
                                        {
                                            '$subtract': [
                                                '$$dividedValue', {
                                                    '$trunc': '$$dividedValue'
                                                }
                                            ]
                                        }, 0.5
                                    ]
                                },
                                'then': {
                                    '$add': [
                                        {
                                            '$trunc': '$$dividedValue'
                                        }, 1
                                    ]
                                },
                                'else': {
                                    '$trunc': '$$dividedValue'
                                }
                            }
                        }
                    }
                },
                'pessimismo': {
                    '$arrayElemAt': [
                        '$respostas.pessimismo', 0
                    ]
                },
                'tristeza': {
                    '$arrayElemAt': [
                        '$respostas.tristeza', 0
                    ]
                },
                'fracasso': {
                    '$arrayElemAt': [
                        '$respostas.fracasso', 0
                    ]
                },
                'prazer': {
                    '$arrayElemAt': [
                        '$respostas.prazer', 0
                    ]
                },
                'culpa': {
                    '$arrayElemAt': [
                        '$respostas.culpa', 0
                    ]
                },
                'punicao': {
                    '$arrayElemAt': [
                        '$respostas.punicao', 0
                    ]
                },
                'estima': {
                    '$arrayElemAt': [
                        '$respostas.estima', 0
                    ]
                },
                'critica': {
                    '$arrayElemAt': [
                        '$respostas.critica', 0
                    ]
                },
                'suicida': {
                    '$arrayElemAt': [
                        '$respostas.suicida', 0
                    ]
                },
                'choro': {
                    '$arrayElemAt': [
                        '$respostas.choro', 0
                    ]
                },
                'agitacao': {
                    '$arrayElemAt': [
                        '$respostas.agitacao', 0
                    ]
                },
                'interesse': {
                    '$arrayElemAt': [
                        '$respostas.interesse', 0
                    ]
                },
                'indecisao': {
                    '$arrayElemAt': [
                        '$respostas.indecisao', 0
                    ]
                },
                'desvalorizacao': {
                    '$arrayElemAt': [
                        '$respostas.desvalorizacao', 0
                    ]
                },
                'energia': {
                    '$arrayElemAt': [
                        '$respostas.energia', 0
                    ]
                },
                'sono': {
                    '$arrayElemAt': [
                        '$respostas.sono', 0
                    ]
                },
                'irritabilidade': {
                    '$arrayElemAt': [
                        '$respostas.irritabilidade', 0
                    ]
                },
                'apetite': {
                    '$arrayElemAt': [
                        '$respostas.apetite', 0
                    ]
                },
                'concentracao': {
                    '$arrayElemAt': [
                        '$respostas.concentracao', 0
                    ]
                },
                'fadiga': {
                    '$arrayElemAt': [
                        '$respostas.fadiga', 0
                    ]
                },
                'int_sexo': {
                    '$arrayElemAt': [
                        '$respostas.int_sexo', 0
                    ]
                },
                'quantAmigos': '$friends.summary.total_count',
                'postMessage': '$posts.message',
                'postStory': '$posts.story',
                'postCreated_time': '$posts.created_time',
                'diaDaSemana': {
                    '$switch': {
                        'branches': [
                            {
                                'case': {
                                    '$eq': [
                                        {
                                            '$dayOfWeek': '$posts.created_time'
                                        }, 1
                                    ]
                                },
                                'then': 'Domingo'
                            }, {
                                'case': {
                                    '$eq': [
                                        {
                                            '$dayOfWeek': '$posts.created_time'
                                        }, 2
                                    ]
                                },
                                'then': 'Segunda-feira'
                            }, {
                                'case': {
                                    '$eq': [
                                        {
                                            '$dayOfWeek': '$posts.created_time'
                                        }, 3
                                    ]
                                },
                                'then': 'Terça-feira'
                            }, {
                                'case': {
                                    '$eq': [
                                        {
                                            '$dayOfWeek': '$posts.created_time'
                                        }, 4
                                    ]
                                },
                                'then': 'Quarta-feira'
                            }, {
                                'case': {
                                    '$eq': [
                                        {
                                            '$dayOfWeek': '$posts.created_time'
                                        }, 5
                                    ]
                                },
                                'then': 'Quinta-feira'
                            }, {
                                'case': {
                                    '$eq': [
                                        {
                                            '$dayOfWeek': '$posts.created_time'
                                        }, 6
                                    ]
                                },
                                'then': 'Sexta-feira'
                            }, {
                                'case': {
                                    '$eq': [
                                        {
                                            '$dayOfWeek': '$posts.created_time'
                                        }, 7
                                    ]
                                },
                                'then': 'Sábado'
                            }
                        ],
                        'default': 'Desconhecido'
                    }
                },
                'hora': {
                    '$hour': '$posts.created_time'
                },
                'minutos': {
                    '$minute': '$posts.created_time'
                },
                'diaDoMes': {
                    '$dayOfMonth': '$posts.created_time'
                },
                'mes': {
                    '$month': '$posts.created_time'
                },
                'ano': {
                    '$year': '$posts.created_time'
                }
            }
        }, {
            '$out': 'postsComBDIAndInfos'
        }
    ]
    collection.aggregate(pipeline3)
    collection = db['postsComBDIAndInfos']

In [ ]:
# Verificando se a coleção existe
if 'postsComBDIAndInfosFiltroDataPosts' in db.list_collection_names():
    print('A coleção já existe')
    collection = db['postsComBDIAndInfosFiltroDataPosts']
else:
    # Primeira parte: Filtrar documentos com 'diaDaSemana' == 'Desconhecido'
    pipeline4 = [
        {
            '$match': {
                'diaDaSemana': {'$ne': 'Desconhecido'}
            }
        },
        {
            '$out': 'postsComBDIAndInfosFiltroDataPosts'
        }
    ]
    collection.aggregate(pipeline4)
    collection = db['postsComBDIAndInfosFiltroDataPosts']

In [ ]:
pipeline5 = [
    {
        '$match': {
            "suicida": {
                "$eq": "3"
            }
        }
    },
    {
        '$group': {
            '_id': '$id_usuario'  # Agrupa por id_usuario
        }
    },
    {
        '$count': 'numero_de_usuarios_unicos'  # Conta quantos grupos únicos foram formados
    }
]

# Executando a agregação
resultado = list(collection.aggregate(pipeline5))

# Obtendo o resultado
if resultado:
    print(f"Usuários únicos: {resultado[0]['numero_de_usuarios_unicos']}")
else:
    print("Nenhum usuário encontrado.")

In [ ]:
# Contando quantos usuarios são do sexo feminino e quantos são do sexo masculino
pipeline6 = [
    {
        '$match': {
            "suicida": {
                "$eq": "3"
            },
            'sexo': 'F'
        }
    },
    {
        '$group': {
            '_id': '$id_usuario'  # Agrupa por id_usuario
        }
    },
    {
        '$count': 'numero_de_usuarios_femininos'
    }
]
quant_feminino = list(collection.aggregate(pipeline6))
print(f"Quantidade de Usuarios Femininos: {quant_feminino[0]['numero_de_usuarios_femininos']}")

pipeline7 = [
    {
        '$match': {
            "suicida": {
                "$eq": "3"
            },
            'sexo': 'M'
        }
    },
    {
        '$group': {
            '_id': '$id_usuario'  # Agrupa por id_usuario
        }
    },
    {
        '$count': 'numero_de_usuarios_masculinos'
    }
]
quant_masculino = list(collection.aggregate(pipeline7))
print(f"Quantidade de Usuarios Masculinos: {quant_masculino[0]['numero_de_usuarios_masculinos']}")

Filtro de 6 meses antes do mês da coleta

In [ ]:
from datetime import datetime, timedelta

if 'posts6Meses' in db.list_collection_names():
    print('A coleção já existe')
    collection = db['posts6Meses']
else:
    data_inicio = datetime(2017, 12, 1)

    data_fim = data_inicio - timedelta(days=6 * 30)

    pipeline8 = [
        {
            '$match': {
                'postCreated_time': {
                    '$gte': data_fim,
                    '$lt': data_inicio
                }
            }
        },
        {
            '$out': 'posts6Meses'
        }
    ]
    collection.aggregate(pipeline8)
    collection = db['posts6Meses']

In [ ]:
# Filtrando documentos com o atributo maior que 3
filtro = {"suicida": {"$eq": "3"}}
documentos = collection.find(filtro)

In [ ]:
# Transformando os documentos em um DataFrame
df = pd.DataFrame(list(documentos))

In [ ]:
df['data'] = pd.to_datetime(df['postCreated_time'])
df.head()

In [ ]:
print(df.dtypes)

In [ ]:
# Lista de colunas que precisam ser convertidas
colunas_para_converter = [
    'pessimismo', 'tristeza', 'fracasso', 'prazer', 'culpa', 'punicao', 'estima',
    'critica', 'suicida', 'choro', 'agitacao', 'interesse', 'indecisao',
    'desvalorizacao', 'energia', 'sono', 'irritabilidade', 'apetite',
    'concentracao', 'fadiga', 'int_sexo', 'quantAmigos'
]

df[colunas_para_converter] = df[colunas_para_converter].astype('int64')

In [ ]:
# Salvar o DataFrame em um arquivo CSV
df.to_csv('dados/filtros_por_data/6meses/so_suicida_6_meses.csv', index=False)

In [ ]:
# filtrar pelo id_usuario
# df = df[df['id_usuario'] == '1022864967872047']

In [ ]:
# Agrupar por usuário, mês e ano
posts_grouped = df.groupby(['id_usuario', 'mes', 'ano']).size().reset_index(name='quantidade')

# Adicionar coluna com o período
posts_grouped['periodo'] = posts_grouped['mes'].astype(str) + '/' + posts_grouped['ano'].astype(str)

In [ ]:
try:
    import matplotlib.pyplot as plt
except ImportError:
    %pip install matplotlib
    import matplotlib.pyplot as plt

In [ ]:
# plotar quantidade de posts por usuario
import matplotlib.pyplot as plt

# Criar o gráfico de linha para cada usuário
for usuario in posts_grouped['id_usuario'].unique():
    df_usuario = posts_grouped[posts_grouped['id_usuario'] == usuario].copy()

    # Convertendo a coluna 'periodo' para datetime para garantir a ordenação correta
    df_usuario['periodo'] = pd.to_datetime(df_usuario['periodo'], format='%m/%Y', errors='coerce')

    # Ordenar os dados por 'periodo'
    df_usuario = df_usuario.sort_values('periodo')

    # Configurar o gráfico de linha
    plt.figure(figsize=(20, 8))  # Aumentar o tamanho da figura

    plt.plot(df_usuario['periodo'].dt.strftime('%m/%Y'), df_usuario['quantidade'], marker='o', linestyle='-',
             color='blue')

    # Adicionar título e rótulos
    plt.title(f'Quantidade de Posts por Mês/Ano - Usuário: {usuario}')
    plt.xlabel('Mês/Ano')
    plt.ylabel('Quantidade de Posts')

    # Melhorar a legibilidade dos rótulos do eixo X
    plt.xticks(rotation=45, ha='right', fontsize=10)

    # Aumentar o pad dos rótulos do eixo X
    plt.gca().tick_params(axis='x', pad=14)  # Aumenta o espaço entre os rótulos e o eixo

    # Adicionar grid
    plt.grid(True, axis='y')

    # Ajustar o layout para evitar sobreposição, com mais padding
    plt.tight_layout(pad=8.0)  # Aumentar o padding geral do layout

    # Adicionar espaço extra ao layout se necessário
    plt.subplots_adjust(bottom=0.2)  # Adiciona mais espaço abaixo dos rótulos do eixo X

    # Salvando o gráfico no diretório dados/filtros_por_data/6meses/graficos
    plt.savefig(f'dados/filtros_por_data/6meses/graficos/quantidade_posts_{usuario}.png')

    # Mostrar o gráfico
    plt.show()

    # Fechar a figura explicitamente para liberar memória
    plt.close()

Efentuando a limpeza dos dados

In [ ]:
try:
    import nltk
except ImportError:
    %pip install nltk
    import nltk

In [ ]:
import re

nltk.download('stopwords')


def remove_caracteres(instancia):
    ''' 
    Função de remoção de caracteres:
    'http\\S+' - remove url 
    lower() - tranforma o texto em minúsculo
    '[0-9]+' - remove números
    '[^\w\s]' -  remove pontuação
    '[!#$%^&*()]' - remove caractéres espaciais
    '''
    instancia = re.sub(r'http\S+', '', instancia).lower()
    instancia = re.sub(r'[0-9]+', '', instancia)
    instancia = re.sub(r'[^\w\s]', '', instancia)
    instancia = re.sub(r'[!#$%^&*()]', '', instancia)
    stopwords = set(nltk.corpus.stopwords.words('portuguese'))
    palavras = [i for i in instancia.split() if not i in stopwords]
    return (' '.join(palavras))


def remove_emojis(string):
    '''Função que remove emojis'''
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)


def remover_ks(texto):
    # Expressão regular para capturar todas as sequências de 'k' (maiúsculo ou minúsculo)
    return re.sub(r'k{2,}', '', texto, flags=re.IGNORECASE)


def remover_js(texto):
    # Expressão regular para capturar todas as sequências de 'j' (maiúsculo ou minúsculo)
    return re.sub(r'j{2,}', '', texto, flags=re.IGNORECASE)

In [ ]:
# aplicando a limpeza dos dados
df['postMessageClean'] = df['postMessage'].fillna('').apply(remove_caracteres).apply(remove_emojis)
df['postMessageClean'] = df['postMessageClean'].apply(remover_ks).apply(remover_js)
pd.set_option('display.max_colwidth', None)
df.head()

In [ ]:
%pip install -U pip setuptools wheel

In [ ]:
try:
    import spacy
    !python -m spacy download pt_core_news_sm
except ImportError:
    %pip install -U spacy
    import spacy
    !python -m spacy download pt_core_news_sm

In [ ]:
nlp = spacy.load('pt_core_news_sm')

In [ ]:
df['postMessageLemma'] = df['postMessageClean'].apply(lambda row: " ".join([w.lemma_ for w in nlp(row)]))
df['postMessageLemma'] = df['postMessageClean'].apply(remove_caracteres)
df.head()
corpus = df['postMessageLemma'].tolist()

In [ ]:
try:
    import numpy as np
except ImportError:
    %pip install -U numpy
    import numpy as np

try:
    from sklearn.feature_extraction.text import TfidfVectorizer
except ImportError:
    %pip install -U scikit-learn
    from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# Definir a lista de stopwords em português
from nltk.corpus import stopwords

stop_words = stopwords.words('portuguese')

custom_stopwords = ['pra', 'pro', 'vc', 'vcs', 'ta', 'tá', 'to', 'tô', 'pq','porque' ,'errada', 'errado', 'entregar', 'vai', 'gonçalves']
stop_words.extend(custom_stopwords)

nomes_df = pd.read_csv('dados/nomes.csv')

# Iterar sobre os valores das colunas 'group_name' e 'first_name' e adicionar os nomes à lista de stopwords
for name in nomes_df['group_name']:
    if pd.notna(name):
        stop_words.append(name.lower())

for name in nomes_df['first_name']:
    if pd.notna(name):
        stop_words.append(name.lower())

for line in nomes_df['alternative_names']:
    if pd.notna(line):
        nomes = line.strip().split('|')
        stop_words.extend([nome.lower() for nome in nomes])

# Criar o vetorizador TF-IDF com parâmetros ajustados
vectorizer = TfidfVectorizer(
    stop_words=stop_words,
    max_features=None,
    max_df=0.40,
    min_df=5,
    ngram_range=(1, 1),
    sublinear_tf=True
)

corpus_tfidf = df['postMessageLemma'].tolist()
tfidf_matrix = vectorizer.fit_transform(corpus_tfidf)

# Obter as palavras
palavras_tfidf = vectorizer.get_feature_names_out()
# Lista para armazenar os resultados
resultados_tfidf = []

# Iterar sobre cada usuário
for usuario in df['id_usuario'].unique():
    # Filtrar os textos do usuário
    indices_usuario = df[df['id_usuario'] == usuario].index
    if len(indices_usuario) > 0:
        # Extrair as palavras com maior score TF-IDF para o usuário
        user_tfidf = tfidf_matrix[indices_usuario]
        user_tfidf_mean = np.asarray(user_tfidf.mean(axis=0)).flatten()

        # Obter os índices das 10 palavras com maior score
        top_10_indices = user_tfidf_mean.argsort()[-10:][::-1]

        # Adicionar as palavras e seus scores à lista de resultados
        for index in top_10_indices:
            resultados_tfidf.append({
                'id_usuario': usuario,
                'palavra': palavras_tfidf[index],
                'score': user_tfidf_mean[index]
            })

# Converter a lista de resultados em DataFrame
resultados_df_tfidf = pd.DataFrame(resultados_tfidf)
resultados_df_tfidf.to_csv('dados/filtros_por_data/6meses/so_suicida_resultados_tfidf_unigramas.csv', index=False)

print(resultados_df_tfidf)

Usando o algoritmo Bag of Words

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(
    stop_words=stop_words,
    ngram_range=(1, 1),
    max_df=0.85,
    min_df=5
)

corpus_bow = df['postMessageLemma'].tolist()
bow_matrix = vectorizer.fit_transform(corpus_bow)

palavras_bow = vectorizer.get_feature_names_out()

resultados_bow = []

for usuario in df['id_usuario'].unique():
    indices_usuario = df[df['id_usuario'] == usuario].index
    if len(indices_usuario) > 0:
        user_bow = bow_matrix[indices_usuario]
        user_bow_sum = np.asarray(user_bow.sum(axis=0)).flatten()

        top_10_indices_bow = user_bow_sum.argsort()[-10:][::-1]

        for index in top_10_indices_bow:
            resultados_bow.append({
                'id_usuario': usuario,
                'palavra': palavras_bow[index],
                'contagem': user_bow_sum[index]
            })

# Converter a lista de resultados em DataFrame
resultados_df_bow = pd.DataFrame(resultados_bow)
resultados_df_bow.to_csv('dados/filtros_por_data/6meses/so_suicida_resultados_bow_unigramas.csv', index=False)

print(resultados_df_bow)